In [51]:
import pandas as pd
import pandas_profiling as pp
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error

import xgboost as xgb
from xgboost import XGBClassifier

In [3]:
train = pd.read_csv("data/train.csv")

/Users/vtrokhymenko/anaconda/envs/py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (8,11,12,13,14,15,16,17,18,19,21,23,26,28,29,30,32,82,83,85,91,269,325,326,327,328) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
train.head()

,CUSTOMER_ID,ACT_TYPE,GENDER,LEGAL_ENTITY,OCCUP_ALL_NEW,AGE,N_CASA_MAX_BALANCE_MTD,N_CASA_MIN_BALANCE_MTD,BL_TAG,TWL_TAG,...,CREDIT_ACT,DEBIT_ACT,CEL_TAG,EL_TAG,AGL_TAG,CVL_TAG,AL_TAG,HL_EMI,SCRUB_EMI,RESPONDERS
0,385793,CSA,M,INDIVIDUAL,SALARIED,21,0.00,0.00,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,N
1,286891,CSA,M,INDIVIDUAL,SALARIED,23,8243.77,7.77,NaN,NaN,...,Y,Y,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,N
2,108751,CSA,M,INDIVIDUAL,SALARIED,41,26515.00,0.00,NaN,NaN,...,Y,Y,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,N
3,278651,CSA,M,INDIVIDUAL,SALARIED,28,65917.28,8270.28,NaN,NaN,...,Y,Y,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,N
4,23637,CSA,M,INDIVIDUAL,SALARIED,34,22260.94,6060.94,NaN,NaN,...,Y,Y,NaN,NaN,NaN,NaN,NaN,NaN,251.758794,N


In [4]:
cat_columns = train.select_dtypes(exclude=[np.number]).columns
object_cols = {}
for col in cat_columns:
        object_cols[col] = 'str'

In [5]:
object_cols

{'ACT_TYPE': 'str', 'GENDER': 'str', 'LEGAL_ENTITY': 'str', 'OCCUP_ALL_NEW': 'str', 'BL_TAG': 'str', 'TWL_TAG': 'str', 'PL_TAG': 'str', 'AL_CNC_TAG': 'str', 'AGRI_TAG': 'str', 'CE_TAG': 'str', 'CV_TAG': 'str', 'EDU_TAG': 'str', 'TL_TAG': 'str', 'OTHER_LOANS_TAG': 'str', 'LAP_TAG': 'str', 'LAS_TAG': 'str', 'BBG_TAG': 'str', 'HL_TAG': 'str', 'GL_TAG': 'str', 'AL_CLOSED': 'str', 'PL_CLOSED': 'str', 'BL_CLOSED': 'str', 'TWL_CLOSED': 'str', 'CV_CLOSED': 'str', 'LAP_CLOSED': 'str', 'LAS_CLOSED': 'str', 'BBG_CLOSED': 'str', 'GL_CLOSED': 'str', 'CC_HOLD_MON_03': 'str', 'CC_HOLD_MON_02': 'str', 'CC_HOLD_MON_01': 'str', 'CC_ACTIVE_MON_03': 'str', 'CC_ACTIVE_MON_02': 'str', 'CC_ACTIVE_MON_01': 'str', 'CC_HOLD': 'str', 'CC_ACTIVE': 'str', 'DC_HOLD': 'str', 'DC_ACTIVE_MON_03': 'str', 'DC_ACTIVE_MON_02': 'str', 'DC_ACTIVE_MON_01': 'str', 'DC_ACTIVE': 'str', 'MATURITY_GL': 'str', 'MATURITY_LAP': 'str', 'MATURITY_LAS': 'str', 'BTE_ACTIVE_MON_01': 'str', 'COC_ACTIVE_MON_01': 'str', 'PL2CC_ACTIVE_MON_01

In [6]:
train = pd.read_csv("data/train.csv", engine='c', sep=',', dtype=object_cols)
y = train["RESPONDERS"]
train.drop("RESPONDERS", axis=1, inplace=True)

test = pd.read_csv("data/test.csv", engine='c', sep=',', dtype=object_cols)

In [11]:
## pp.ProfileReport(train[train.columns[:100]])

/Users/vtrokhymenko/anaconda/envs/py36/lib/python3.6/site-packages/pandas_profiling/base.py:223: RuntimeWarning: divide by zero encountered in long_scalars
  'p_unique': distinct_count / count}
/Users/vtrokhymenko/anaconda/envs/py36/lib/python3.6/site-packages/pandas_profiling/base.py:223: RuntimeWarning: divide by zero encountered in long_scalars
  'p_unique': distinct_count / count}
/Users/vtrokhymenko/anaconda/envs/py36/lib/python3.6/site-packages/pandas_profiling/base.py:223: RuntimeWarning: divide by zero encountered in long_scalars
  'p_unique': distinct_count / count}


KeyboardInterrupt: 

In [7]:
numeric_cols = train.select_dtypes(include=[np.number]).columns
numeric_cols = [col for col in numeric_cols if col != 'CUSTOMER_ID']

In [8]:
numeric_cols

['AGE',
 'N_CASA_MAX_BALANCE_MTD',
 'N_CASA_MIN_BALANCE_MTD',
 'EEG_TAG',
 'EEG_CLOSED',
 'DC_SPEND_MON_03',
 'DC_SPEND_MON_02',
 'DC_SPEND_MON_01',
 'CC_SPEND_MON_04',
 'CC_SPEND_MON_01',
 'CC_SPEND_MON_02',
 'CC_SPEND_MON_03',
 'DC_SPEND_MON_04',
 'PM_FD_MON_01',
 'PM_FD_MON_02',
 'PM_FD_MON_04',
 'CR_LIMIT',
 'STMT_MON_03',
 'STMT_MON_02',
 'STMT_MON_01',
 'COC_ELIGIBLE',
 'LAP_SCRUB_LIVE',
 'LAS_SCRUB_LIVE',
 'PL_SCRUB_LIVE',
 'GL_SCRUB_LIVE',
 'TWL_SCRUB_LIVE',
 'BL_SCRUB_LIVE',
 'CEL_SCRUB_LIVE',
 'TL_SCRUB_LIVE',
 'AL_SCRUB_LIVE',
 'LAP_SCRUB_CLOSED',
 'LAS_SCRUB_CLOSED',
 'PL_SCRUB_CLOSED',
 'GL_SCRUB_CLOSED',
 'TWL_SCRUB_CLOSED',
 'BL_SCRUB_CLOSED',
 'CEL_SCRUB_CLOSED',
 'TL_SCRUB_CLOSED',
 'AL_SCRUB_CLOSED',
 'STMT_CON_DAE_ACTIVE_MON_01',
 'STMT_CON_DAE_CLOSED_MON_01',
 'MER_EMI_CLOSED_MON_01',
 'EOP_BAL_MON_01',
 'AMB_MON_04',
 'AMB_MON_01',
 'AMB_MON_02',
 'AMB_MON_03',
 'NB_MON_01_CNT',
 'NB_MON_02_CNT',
 'NB_MON_03_CNT',
 'CDMAT_MON_01',
 'CDMAT_MON_04',
 'MF_MON_04',
 'M

In [9]:
cat_columns = train.select_dtypes(exclude=[np.number]).columns
cat_columns

Index(['ACT_TYPE', 'GENDER', 'LEGAL_ENTITY', 'OCCUP_ALL_NEW', 'BL_TAG',
       'TWL_TAG', 'PL_TAG', 'AL_CNC_TAG', 'AGRI_TAG', 'CE_TAG', 'CV_TAG',
       'EDU_TAG', 'TL_TAG', 'OTHER_LOANS_TAG', 'LAP_TAG', 'LAS_TAG', 'BBG_TAG',
       'HL_TAG', 'GL_TAG', 'AL_CLOSED', 'PL_CLOSED', 'BL_CLOSED', 'TWL_CLOSED',
       'CV_CLOSED', 'LAP_CLOSED', 'LAS_CLOSED', 'BBG_CLOSED', 'GL_CLOSED',
       'CC_HOLD_MON_03', 'CC_HOLD_MON_02', 'CC_HOLD_MON_01',
       'CC_ACTIVE_MON_03', 'CC_ACTIVE_MON_02', 'CC_ACTIVE_MON_01', 'CC_HOLD',
       'CC_ACTIVE', 'DC_HOLD', 'DC_ACTIVE_MON_03', 'DC_ACTIVE_MON_02',
       'DC_ACTIVE_MON_01', 'DC_ACTIVE', 'MATURITY_GL', 'MATURITY_LAP',
       'MATURITY_LAS', 'BTE_ACTIVE_MON_01', 'COC_ACTIVE_MON_01',
       'PL2CC_ACTIVE_MON_01', 'MER_EMI_ACTIVE_MON_01',
       'TRN_CON_DAE_ACTIVE_MON_01', 'BTE_CLOSED_MON_01', 'COC_CLOSED_MON_01',
       'PL2CC_CLOSED_MON_01', 'TRN_CON_DAE_CLOSED_MON_01', 'FINANCE_MON_01',
       'JOBS_MON_01', 'LIFESTYLE_MON_01', 'INMON_01KET_MON_01',

In [10]:
%%time

from collections import defaultdict
d = defaultdict(LabelEncoder)

# fit and encode train/test
a = pd.concat([train[cat_columns], test[cat_columns]], axis=0).fillna('').apply(lambda x: d[x.name].fit(x))

# transform encodings to train
train[cat_columns] = train[cat_columns].fillna('').apply(lambda x: d[x.name].transform(x))

# transform encodings to test
test[cat_columns] = test[cat_columns].fillna('').apply(lambda x: d[x.name].transform(x))

CPU times: user 1min 28s, sys: 8.3 s, total: 1min 36s
Wall time: 1min 39s


In [11]:
train[cat_columns].head()

,ACT_TYPE,GENDER,LEGAL_ENTITY,OCCUP_ALL_NEW,BL_TAG,TWL_TAG,PL_TAG,AL_CNC_TAG,AGRI_TAG,CE_TAG,...,TOP_100_CORP,TOP_800_CORP,TOP_CORP_TAG,CREDIT_ACT,DEBIT_ACT,CEL_TAG,EL_TAG,AGL_TAG,CVL_TAG,AL_TAG
0,0,2,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,2,2,0,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,0,0
2,0,2,2,0,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,0,0
3,0,2,2,0,0,0,1,0,0,0,...,0,0,0,1,1,0,0,0,0,0
4,0,2,2,0,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,0,0


In [12]:
y.value_counts(normalize=True)

N    0.984627
Y    0.015373
Name: RESPONDERS, dtype: float64

In [13]:
y = y.map({"Y": 1, "N": 0})

In [30]:
import lightgbm as lgb

params = {
    'application': 'binary',
    'num_leaves': 255,
    'feature_fraction': 0.6,
    'sub_row': 0.6,
    'bagging_freq': 1,
    'max_bin': 255,
    'metric': 'auc',
    'num_threads': 4,
    'verbose': 0,
    'seed': 42
}

#b'Number of classes must be 1 for non-multiclass training'

num_rounds = 1000

In [163]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
validation = []
for train_indexes, val_indexes in skf.split(train, y):
    validation.append((train_indexes, val_indexes))

In [15]:
train_cols = numeric_cols + list(cat_columns)

In [16]:
lgb_train = lgb.Dataset(train[train_cols], y)

In [17]:
len(train[train_cols].columns)

331

---

In [38]:
X_train_xgb, X_test_xgb, y_train_xgb, y_test_xgb = train_test_split(train[train_cols], y,
                                                    test_size=0.2, random_state=42)

In [36]:
gbm = lgb.LGBMRegressor(objective='regression',
                        num_leaves=31,
                        learning_rate=0.05,
                        n_estimators=20)

In [39]:
%%time
gbm.fit(X_train_xgb, y_train_xgb,
        eval_set=[(X_test_xgb, y_test_xgb)],
        eval_metric='auc',
        early_stopping_rounds=15)

[1]	valid_0's auc: 0.836456
Training until validation scores don't improve for 15 rounds.
[2]	valid_0's auc: 0.850021
[3]	valid_0's auc: 0.854964
[4]	valid_0's auc: 0.859338
[5]	valid_0's auc: 0.860711
[6]	valid_0's auc: 0.861205
[7]	valid_0's auc: 0.866581
[8]	valid_0's auc: 0.867087
[9]	valid_0's auc: 0.872039
[10]	valid_0's auc: 0.871076
[11]	valid_0's auc: 0.872452
[12]	valid_0's auc: 0.874488
[13]	valid_0's auc: 0.87516
[14]	valid_0's auc: 0.875434
[15]	valid_0's auc: 0.876508
[16]	valid_0's auc: 0.876318
[17]	valid_0's auc: 0.879306
[18]	valid_0's auc: 0.878779
[19]	valid_0's auc: 0.879795
[20]	valid_0's auc: 0.879648
CPU times: user 24.4 s, sys: 6.19 s, total: 30.6 s
Wall time: 17.6 s


LGBMRegressor(boosting_type='gbdt', colsample_bytree=1, learning_rate=0.05,
       max_bin=255, max_depth=-1, min_child_samples=10, min_child_weight=5,
       min_split_gain=0, n_estimators=20, nthread=-1, num_leaves=31,
       objective='regression', reg_alpha=0, reg_lambda=0, seed=0,
       silent=True, subsample=1, subsample_for_bin=50000, subsample_freq=1)

In [41]:
y_gbm = gbm.predict(X_test_xgb, num_iteration=gbm.best_iteration)

In [46]:
print('The rmse of prediction is:', mean_squared_error(y_test_xgb, y_gbm) ** 0.5)

The rmse of prediction is: 0.116320860619


In [48]:
print('Feature importances:', list(gbm.feature_importances_))

Feature importances: [0, 0, 5, 0, 0, 1, 2, 1, 3, 8, 3, 0, 3, 0, 0, 0, 28, 3, 3, 9, 2, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 2, 2, 2, 6, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 3, 1, 0, 2, 1, 1, 13, 0, 1, 0, 1, 4, 0, 0, 3, 0, 0, 0, 0, 3, 0, 3, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 2, 3, 2, 9, 1, 3, 0, 3, 1, 4, 3, 1, 3, 5, 4, 2, 0, 1, 1, 4, 2, 0, 0, 1, 9, 2, 0, 4, 6, 2, 0, 0, 0, 2, 0, 4, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 11, 6, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 18, 8, 10, 1, 1, 0, 13, 0, 0, 0, 0, 0, 36, 39, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 16, 0, 0, 0, 0, 0, 0, 0, 3, 15, 14, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 18, 0, 0, 0, 0, 15, 53, 0, 0, 26, 0, 0, 1, 0, 0, 18, 3, 0, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [49]:
estimator = lgb.LGBMRegressor(num_leaves=31)

In [52]:
%%time
param_grid = {
    'learning_rate': [0.01, 0.1, 1],
    'n_estimators': [20, 40]
}

gbm = GridSearchCV(estimator, param_grid)

gbm.fit(X_train_xgb, y_train_xgb)

GridSearchCV(cv=None, error_score='raise',
       estimator=LGBMRegressor(boosting_type='gbdt', colsample_bytree=1, learning_rate=0.1,
       max_bin=255, max_depth=-1, min_child_samples=10, min_child_weight=5,
       min_split_gain=0, n_estimators=10, nthread=-1, num_leaves=31,
       objective='regression', reg_alpha=0, reg_lambda=0, seed=0,
       silent=True, subsample=1, subsample_for_bin=50000, subsample_freq=1),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'learning_rate': [0.01, 0.1, 1], 'n_estimators': [20, 40]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [53]:
print('Best parameters found by grid search are:', gbm.best_params_)

Best parameters found by grid search are: {'learning_rate': 0.1, 'n_estimators': 40}


In [54]:
gbm = lgb.LGBMRegressor(objective='regression',
                        num_leaves=31,
                        learning_rate=0.1,
                        n_estimators=40)

In [58]:
%%time
gbm.fit(train[train_cols], y)

CPU times: user 37.8 s, sys: 6.57 s, total: 44.4 s
Wall time: 22 s


LGBMRegressor(boosting_type='gbdt', colsample_bytree=1, learning_rate=0.1,
       max_bin=255, max_depth=-1, min_child_samples=10, min_child_weight=5,
       min_split_gain=0, n_estimators=40, nthread=-1, num_leaves=31,
       objective='regression', reg_alpha=0, reg_lambda=0, seed=0,
       silent=True, subsample=1, subsample_for_bin=50000, subsample_freq=1)

In [62]:
gbm.predict(test[train_cols], num_iteration=gbm.best_iteration)

array([ 0.01821538,  0.00851788,  0.00120457, ...,  0.02765422,
        0.00164516,  0.14846008])

In [ ]:
y_gbm

In [63]:
test["RESPONDERS"] = gbm.predict(test[train_cols], num_iteration=gbm.best_iteration)

In [64]:
test[submission.columns].to_csv("predictions/baseline-gbm0.csv", index=False)

In [65]:
pd.read_csv('predictions/baseline-gbm0.csv').head(10)

,CUSTOMER_ID,RESPONDERS
0,15800,0.018215
1,463196,0.008518
2,309959,0.001205
3,102348,0.001205
4,114850,0.022095
5,297584,0.001645
6,50514,0.013919
7,440096,0.001645
8,486760,0.001645
9,430986,0.012623


---

---

In [126]:
dtrain = xgb.DMatrix(train[train_cols], label=y)
dtest = xgb.DMatrix(test[train_cols])

In [128]:
param = {
    'max_depth': 2,  # the maximum depth of each tree
    'eta': 0.5,  # the training step for each iteration
    'silent': 1,  # logging mode - quiet
    'num_class': 2}  # the number of classes that exist in this datset
num_round = 10  # the number of training iterations

In [129]:
%%time
bst = xgb.train(param, dtrain, num_round)

CPU times: user 48.5 s, sys: 395 ms, total: 48.9 s
Wall time: 13.1 s


In [195]:
X_train_xgb, X_test_xgb, y_train_xgb, y_test_xgb = train_test_split(train[train_cols], y,
                                                    test_size=0.2, random_state=42)

In [196]:

model = XGBClassifier()
model.fit(X_train_xgb, y_train_xgb)

CPU times: user 4min 33s, sys: 4.73 s, total: 4min 37s
Wall time: 1min 35s


In [199]:
y_pred = model.predict(X_test_xgb)
y_pred

array([0, 0, 0, ..., 0, 0, 0])

In [200]:
accuracy = accuracy_score(y_test_xgb, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 98.49%


In [240]:
%%time
model_new = XGBClassifier(gamma=0.8,max_depth=5, n_estimators=300,colsample_bytree=1,min_child_weight=3,seed=27)
model_new.fit(train[train_cols], y)

CPU times: user 1h 16min, sys: 41.7 s, total: 1h 16min 42s
Wall time: 23min 44s


In [241]:
y_pred_new=model_new.predict_proba(test[train_cols])

In [24]:
test.drop('RESPONDERS',axis=1,inplace=True)

In [243]:
test["RESPONDERS"] = y_pred_new[:,1]

In [244]:
test[submission.columns].to_csv("predictions/pred-XGB2.csv", index=False)

In [245]:
pd.read_csv('predictions/pred-XGB2.csv').head()

,CUSTOMER_ID,RESPONDERS
0,15800,0.002443
1,463196,0.001740
2,309959,0.000312
3,102348,0.000365
4,114850,0.010249


---

In [171]:
lgb_train = lgb.Dataset(train[train_cols], y)
cv = lgb.cv(params, lgb_train, num_rounds, folds=validation, early_stopping_rounds=15, verbose_eval=1)

[1]	cv_agg's auc: 0.799405 + 0.00926388
[2]	cv_agg's auc: 0.819788 + 0.00715325
[3]	cv_agg's auc: 0.831816 + 0.00477732
[4]	cv_agg's auc: 0.853022 + 0.00509706
[5]	cv_agg's auc: 0.854064 + 0.00407079
[6]	cv_agg's auc: 0.854032 + 0.00362619
[7]	cv_agg's auc: 0.856974 + 0.00421452
[8]	cv_agg's auc: 0.857586 + 0.00353799
[9]	cv_agg's auc: 0.858347 + 0.00322677
[10]	cv_agg's auc: 0.863576 + 0.00441697
[11]	cv_agg's auc: 0.868538 + 0.00438192
[12]	cv_agg's auc: 0.867916 + 0.00431923
[13]	cv_agg's auc: 0.868356 + 0.00420466
[14]	cv_agg's auc: 0.869378 + 0.0033523
[15]	cv_agg's auc: 0.870136 + 0.00254746
[16]	cv_agg's auc: 0.870268 + 0.002307
[17]	cv_agg's auc: 0.870579 + 0.00272195
[18]	cv_agg's auc: 0.871781 + 0.00291614
[19]	cv_agg's auc: 0.87178 + 0.00277957
[20]	cv_agg's auc: 0.871453 + 0.00262834
[21]	cv_agg's auc: 0.871464 + 0.00277297
[22]	cv_agg's auc: 0.870685 + 0.00300123
[23]	cv_agg's auc: 0.874806 + 0.00362314
[24]	cv_agg's auc: 0.874879 + 0.00391375
[25]	cv_agg's auc: 0.876934 +

In [173]:
def custom_metric_lift(df):
    return df.sort_values('PROBABILITY', ascending=False).iloc[:int(len(temp_df)*0.1), 1].sum() / df["RESPONDERS"].sum()

In [174]:
scores = []
for train_indexes, val_indexes in validation:
    temp_df = pd.concat([train.loc[val_indexes,["CUSTOMER_ID"]],y.loc[val_indexes]], axis=1)
    model = lgb.train(params, lgb_train.subset(train_indexes), 94)
    temp_df["PROBABILITY"] = model.predict(train.loc[val_indexes, train_cols])
    scores.append(custom_metric_lift(temp_df))
    print("Score is {}".format(scores[-1]))

Score is 0.6652221018418202
Score is 0.6424702058504875
Score is 0.658351409978308
Score is 0.6626898047722343
Score is 0.6832971800433839


In [175]:
np.mean(scores), np.std(scores)

(0.66240614049724678, 0.013102121712565267)

---

In [131]:
preds = bst.predict(dtest)
preds

array([ 0.,  0.,  0., ...,  0.,  0.,  0.], dtype=float32)

In [94]:
test["RESPONDERS"] = preds

In [95]:
test[submission.columns].to_csv("predictions/baseline1.csv", index=False)

---

In [60]:
test.drop('RESPONDERS',axis=1,inplace=True)

# Predict test

In [32]:
%%time
model = lgb.train(params, lgb_train, num_boost_round=91)

CPU times: user 32.1 s, sys: 507 ms, total: 32.6 s
Wall time: 8.62 s


In [26]:
submission = pd.read_csv("data/sample_submission.csv")

In [33]:
test["RESPONDERS"] = model.predict(test[train_cols])

In [34]:
test[submission.columns].to_csv("predictions/baseline-lgb2.csv", index=False)

In [35]:
pd.read_csv('predictions/baseline-lgb2.csv').head(10)

,CUSTOMER_ID,RESPONDERS
0,15800,0.005510
1,463196,0.007526
2,309959,0.000865
3,102348,0.000778
4,114850,0.015960
5,297584,0.001848
6,50514,0.006219
7,440096,0.001175
8,486760,0.001197
9,430986,0.012126
